There was no way to batch download images off Civitai and that really bugged me so I made this! I can't take all the credit, I did use ChatGPT to do this as coding just makes me want to yeet myself out a building window.

**Nothing gets sent to ChatGPT**, **nothing gets sent to me**, this just saves your Civitai images to your Google Drive. Google's data handling policies and privacy policies obviously apply here.

Below you'll need to hit the "play" or run button on these cells **in order**. The first cell will ask you for 3 things - your Civitai username (this is to ensure it downloads your images and not all the images on the site or someone else's images), your API key (you can access that at http://https://civitai.com/user/account), and the number of images to download (you can find this on your profile and going across the top menu that displays Models, Posts, Images, etc.) if you don't define the number of images then it will just keep downloading until either **Civitai bans you from the API** or **Google stops giving you compute power** so including the number of images you're downloading is important. **This tool cannot download individual images in any kind of predicatble way, this is only for batch downloading all your images at once**.

# **User Inputs (ONLY CHANGE THESE TO SUIT YOUR OWN USERNAME, API KEY, AND IMAGES)**

Click the run button and a text box will become visible to ask you for the following inputs;

In [1]:
# User inputs
civitai_username = input("Enter your Civitai username: ")
api_key = input("Enter your Civitai API key: ")
max_images = int(input("Enter the number of images to download: "))

Enter the number of images to download: 3533


# **Mount Google Drive**

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Install Required Libraries**

In [3]:
!pip install requests tenacity tqdm

# **Error Logging**

In [4]:
import logging
import os
import requests
import time
from tenacity import retry, stop_after_attempt, wait_fixed, RetryError

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# **Define Helper Functions**

In [5]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
def fetch_url(url, headers, params=None):
    logging.info(f"Fetching URL: {url} with params: {params}")
    response = requests.get(url, headers=headers, params=params)
    logging.info(f"Response Status Code: {response.status_code}")
    if response.status_code != 200:
        logging.error(f"Response Content: {response.content}")
    response.raise_for_status()
    return response

# **Set Up API and Directory**

In [6]:
# API Endpoint and API key
API_ENDPOINT = "https://civitai.com/api/v1/images"
SAVE_DIR = "/content/drive/MyDrive/Civitai Downloads"

# Create directory to save images
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# **Download Function**

In [7]:
def download_images(civitai_username, api_key, max_images):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    params = {
        "username": civitai_username,  # Using the specified username
        "sort": "Newest",
        "nsfw": "X"
    }

    image_counter = 0
    next_page = API_ENDPOINT

    while next_page and image_counter < max_images:
        try:
            response = fetch_url(next_page, headers, params)
            data = response.json()
            logging.info(f"Fetched page: {next_page}")
            images = data.get("items", [])
            if not images:
                logging.error("No images found in response")
                break

            for image in images:
                if image_counter >= max_images:
                    break
                image_url = image["url"]
                image_name = f"{civitai_username}_civ({image_counter + 1}).jpg"
                image_path = os.path.join(SAVE_DIR, image_name)

                # Check if the image already exists
                if os.path.exists(image_path):
                    logging.info(f"Skipping {image_name}, already exists.")
                    continue

                try:
                    image_response = fetch_url(image_url, headers)
                    with open(image_path, 'wb') as file:
                        file.write(image_response.content)
                    logging.info(f"Downloaded {image_name}")
                    image_counter += 1
                    print(f"Downloaded {image_counter} of {max_images} images.")
                except RetryError as e:
                    logging.error(f"Failed to download {image_name} after retries: {e}")
                except requests.HTTPError as e:
                    logging.error(f"HTTP error while downloading {image_name}: {e}")
                    if e.response.status_code == 429:  # Handle rate limit error
                        logging.error("Rate limit exceeded. Waiting before retrying...")
                        time.sleep(60)  # Wait for a minute before retrying
                    else:
                        break
                except Exception as e:
                    logging.error(f"An error occurred while downloading {image_name}: {e}")

            next_cursor = data.get("metadata", {}).get("nextCursor")
            if next_cursor:
                params["cursor"] = next_cursor
            else:
                logging.info("No more pages to fetch.")
                break
            time.sleep(1)  # Rate limiting: sleep for 1 second between page requests
        except RetryError as e:
            logging.error(f"Failed to fetch images after retries: {e}")
            break
        except requests.HTTPError as e:
            logging.error(f"HTTP error occurred: {e.response.status_code} - {e.response.text}")
            if e.response.status_code == 429:
                logging.error("Rate limit exceeded. Waiting before retrying...")
                time.sleep(60)
            else:
                break
        except Exception as e:
            logging.error(f"An error occurred: {e}")
            break

# **Download your Images**

In [8]:
if __name__ == "__main__":
    download_images(civitai_username, api_key, max_images)

Downloaded 1 of 3533 images.
Downloaded 2 of 3533 images.
Downloaded 3 of 3533 images.
Downloaded 4 of 3533 images.
Downloaded 5 of 3533 images.
Downloaded 6 of 3533 images.
Downloaded 7 of 3533 images.
Downloaded 8 of 3533 images.
Downloaded 9 of 3533 images.
Downloaded 10 of 3533 images.
Downloaded 11 of 3533 images.
Downloaded 12 of 3533 images.
Downloaded 13 of 3533 images.
Downloaded 14 of 3533 images.
Downloaded 15 of 3533 images.
Downloaded 16 of 3533 images.
Downloaded 17 of 3533 images.
Downloaded 18 of 3533 images.
Downloaded 19 of 3533 images.
Downloaded 20 of 3533 images.
Downloaded 21 of 3533 images.
Downloaded 22 of 3533 images.
Downloaded 23 of 3533 images.
Downloaded 24 of 3533 images.
Downloaded 25 of 3533 images.
Downloaded 26 of 3533 images.
Downloaded 27 of 3533 images.
Downloaded 28 of 3533 images.
Downloaded 29 of 3533 images.
Downloaded 30 of 3533 images.
Downloaded 31 of 3533 images.
Downloaded 32 of 3533 images.
Downloaded 33 of 3533 images.
Downloaded 34 of 35